In [2]:
from src.config import filepaths
from src.utils import construct_sparse_adj_mat, load_json
import numpy as np
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
from scipy.stats import entropy

In [3]:
smfp = filepaths['artifacts_sim_mats']
dataset = 'sprhea'
toc = 'v3_folded_pt_ns'

adj, idx_sample, idx_feature = construct_sparse_adj_mat(dataset, toc)
krs = load_json(filepaths['data_sprhea'] / f"{toc}.json")
rules = pd.read_csv(filepaths['data'] / "minimal1224_all_uniprot.tsv", sep='\t', )
rules.set_index('Name', inplace=True)

sim_metrics = [
    'bag_of_molecules_proteins_x_reactions',

]

sim_mats = {name.removesuffix("_proteins_x_reactions"): np.load(smfp / f"{dataset}_{toc}_{name}.npy")
                for name in sim_metrics}

Constructing sprhea:v3_folded_pt_ns sparse adjacency matrix


In [4]:
# def axwise_entropy(mat: np.ndarray, axis: int):
#     if axis != 0 and axis != 1:
#         raise ValueError("Invalid axis. Only 2D arrays supported")
#     elif axis == 0:
#         axsum = mat.sum(axis=axis).reshape(1, -1)
#     elif axis == 1:
#         axsum = mat.sum(axis=axis).reshape(-1, 1)
    
#     return np.where(axsum != 0, mat / axsum, 0)

# prot_entropy = axwise_entropy(sim_mats['bag_of_molecules'], axis=1)
# rxn_entropy = axwise_entropy(sim_mats['bag_of_molecules'], axis=0)
# prot_entropy.max(), prot_entropy.min()

In [6]:
diff = sim_mats['bag_of_molecules'] - adj

cross_overs = np.argwhere(diff > 0.5)
cross_overs = np.vstack(sorted(cross_overs, key=lambda x: diff[*x], reverse=True))

: 

: 

: 

In [1]:
for i, j in cross_overs:
    print(idx_sample[i], idx_feature[j])
    print(diff[i, j])


KeyboardInterrupt

